# Generating C++ Kokkos Chemistry Solvers with JAFF

This notebook demonstrates how to use JAFF to generate C++ code that uses the Kokkos-kernels BDF solver for solving chemical reaction networks. We'll use the `test2.dat` network as an example, which contains only 3 species and 2 reactions.

## Overview

The Kokkos output plugin generates:
- **chemistry_ode.hpp**: Chemistry ODE system class with evaluate_function and evaluate_jacobian methods
- **chemistry_ode.cpp**: Main executable that sets up and runs the BDF solver
- **CMakeLists.txt**: Build configuration for Kokkos and Kokkos-kernels dependencies

The generated code is compatible with the Kokkos-kernels BDF solver for stiff ODEs.

## Step 1: Load the Chemical Network

First, let's load the react_COthin network, which is a carbon-oxygen chemistry network in KROME format.

In [1]:
from jaff.network import Network
from jaff.builder import Builder
import os

# Load the test2 network
network = Network("../networks/react_COthin")

print(f"Network loaded: {network.label}")
print(f"Number of species: {network.get_number_of_species()}")
print(f"Number of reactions: {len(network.reactions)}")

Welcome to JAFF: Just Another Fancy Format!
Loading network from ../networks/react_COthin
Network label = react_COthin


  8%|████████▏                                                                                                 | 27/350 [00:00<00:01, 192.49it/s]

KROME variable detected: @var:Hnuclei = get_Hnuclei(n(:))
KROME variable detected: @var: Te = Tgas*8.617343d-5
KROME variable detected: @var: invT = 1d0/Tgas
KROME variable detected: @var: lnTe = log(Te)
KROME variable detected: @var: T = Tgas
KROME format detected: @format:idx,R,R,R,P,P,P,P,Tmin,Tmax,rate
exp(-2.03914985e-6*lnTe**8 + 0.000111954395*lnTe**7 - 0.00263197617*lnTe**6 + 0.0348255977*lnTe**5 - 0.2877056*lnTe**4 + 1.56315498*lnTe**3 - 5.73932875*lnTe**2 + 13.536556*lnTe - 32.71396786)
3.92e-13*invTe**0.6353
exp(-3.071135243196595e-9*lnTe**9 - 1.856767039775261e-8*lnTe**8 + 5.755614137575758e-7*lnTe**7 + 4.989108920299513e-6*lnTe**6 - 1.421502914054107e-5*lnTe**5 - 0.0003212605213188796*lnTe**4 - 0.002380861877349834*lnTe**3 - 0.02026044731984691*lnTe**2 - 0.7241125657826851*lnTe - 28.61303380689232)
exp(-3.64916141e-6*lnTe**8 + 0.000206723616*lnTe**7 - 0.0050090561*lnTe**6 + 0.0679539123*lnTe**5 - 0.56851189*lnTe**4 + 3.05803875*lnTe**3 - 10.7532302*lnTe**2 + 23.91596563*lnT

 14%|██████████████▊                                                                                           | 49/350 [00:00<00:01, 204.87it/s]

2.56e-9*Te**1.78186
exp(-8.06838246118e-8*lnTe**9 + 2.4555011970392e-6*lnTe**8 - 2.585009680264e-5*lnTe**7 + 8.66396324309e-5*lnTe**6 + 0.0002012250284791*lnTe**5 - 0.0014327641212992*lnTe**4 + 0.00846445538663*lnTe**3 - 0.1421013521554148*lnTe**2 + 1.139449335841631*lnTe - 20.37260896533324)
6.5e-9/(sqrt(Te))
1.0e-8/Tgas**0.4
1.00000000000000e-8
1.32e-6/Tgas**0.76
5.0e-7*sqrt(100.0*invT)
KROME variable detected: @var:kl21 = 1.18d-10*exp(-6.95d4*invT)
KROME variable detected: @var:kh21 = 8.125d-8*invsqrT*exp(-5.2d4*invT)*(1.d0-exp(-6d3*invT))
KROME variable detected: @var:ncr21 = 1d1**(4.845d0-1.3d0*log10(T*1d-4)+1.62d0*log10(T*1d-4)**2)
KROME variable detected: @var:a21=1.d0/(1.d0+(Hnuclei/(ncr21+1d-40)))
kh21**(1.0 - a21)*kl21**a21


...

1.0e-9*exp(-7080.0*invT)
1.60000000000000e-9
7.50000000000000e-10
7.0e-10*exp(-10560.0*invT)
2.50000000000000e-10
2.50000000000000e-10
4.80000000000000e-10
1.69000000000000e-9
1.50000000000000e-9
7.98e-10*exp(-1.41*invT)/T32**0.156
3.42e-10*exp(-1.41*invT)/T32**0.156
2.28000000000000e-8
1.52e-9*(2.62185*(300.0*invT)**0.5 + 0.62)
2.28000000000000e-8
9.15e-10*(2.62185*(300.0*invT)**0.5 + 0.62)
1.01000000000000e-9
6.40000000000000e-10
2.55000000000000e-8
1.73e-9*(2.578947*(300.0*invT)**0.5 + 0.62)
1.80000000000000e-9
5.03000000000000e-9
3.4093e-10*(2.578947*(300.0*invT)**0.5 + 0.62)
2.40000000000000e-10
1.00000000000000e-11
3.42000000000000e-10
4.53000000000000e-10
9.10000000000000e-10
5.20000000000000e-11
5.20000000000000e-11
2.47000000000000e-9
1.88055e-9*(1.79558*invT + 0.02427*(300.0*invT)**0.5 + 1.0)
1.45000000000000e-10
1.08256e-10*(1.79558*invT + 0.02427*(300.0*invT)**0.5 + 1.0)
1.00000000000000e-9
1.23000000000000e-8
8.34e-10*(834.16588*invT + 0.5232*(300.0*invT)**0.5 + 1.0)
3.300

 76%|████████████████████████████████████████████████████████████████████████████████                         | 267/350 [00:00<00:00, 573.56it/s]

1.50000000000000e-15
9.9e-19/T32**0.38
4.9e-20*T32**1.58
5.26e-18*exp(-90.0*invT)/T32**5.22
KROME format detected: @format:idx,R,R,P,rate
KROME variable detected: @var:fA = 1d0/(1d0+1d4*exp(-6d2/(user_Tdust+1d-40)))
3.0e-18*Hnuclei*sqrt(Tgas)*fA/((8.0e-6*Tgas**2 + 0.002*Tgas + 0.04*sqrt(Tgas + user_Tdust) + 1.0)*n(idx_H))
KROME format detected: @format:idx,R,P,P,rate
7.1e-7*exp(-0.5*user_Av)
1.1e-9*exp(-1.9*user_Av)
KROME variable detected: @var: user_H2self = fselfH2(1.87d21*(merge(n(idx_H2),n_global(idx_H2),n(idx_H2) > 0.0_8)*1d-3)**(2./3.), 1d5)
5.6e-11*user_H2self*exp(-3.74*user_Av)
4.9e-13*exp(-1.8*user_Av)
4.9e-13*exp(-2.3*user_Av)
3.1e-10*exp(-3.0*user_Av)
2.4e-7*exp(-0.9*user_Av)
8.7e-10*exp(-1.2*user_Av)
7.7e-10*exp(-2.8*user_Av)
2.6e-10*exp(-2.5*user_Av)
7.1e-10*exp(-1.7*user_Av)
5.9e-10*exp(-2.3*user_Av)
4.6e-10*exp(-1.7*user_Av)
1.0e-9*exp(-1.7*user_Av)
1.0e-9*exp(-1.7*user_Av)
1.5e-10*exp(-2.1*user_Av)
2.4e-7*exp(-0.5*user_Av)
3.7e-10*exp(-1.7*user_Av)
1.6e-12*exp(-3.1*use

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 350/350 [00:00<00:00, 456.14it/s]

Variables found: ['HnOj', 'Hnuclei', 'T', 'T32', 'Te', 'Tgas', 'crate', 'fA', 'idx_H', 'invT', 'invT32', 'invTe', 'invTgas', 'lnTe', 'n', 'ntot', 'sqrTgas', 'tgas', 'user_Av', 'user_H2self', 'user_Tdust']
Loaded 287 reactions
Lodaded 0 photo-chemistry reactions
All done!
Network loaded: react_COthin
Number of species: 37
Number of reactions: 287


## Step 2: Examine Network Properties

Let's examine some properties of the network to understand what we're working with.

In [2]:
for i, species in enumerate(network.species):
    print(f"  {i}: {species.name} (mass: {species.mass:.2f}, charge: {species.charge})")

print(f"\nTotal species: {len(network.species)}")

  0: H (mass: 0.00, charge: 0)
  1: e- (mass: 0.00, charge: -1)
  2: H+ (mass: 0.00, charge: 1)
  3: He (mass: 0.00, charge: 0)
  4: He+ (mass: 0.00, charge: 1)
  5: He++ (mass: 0.00, charge: 2)
  6: H2 (mass: 0.00, charge: 0)
  7: H2+ (mass: 0.00, charge: 1)
  8: H- (mass: 0.00, charge: -1)
  9: C+ (mass: 0.00, charge: 1)
  10: C (mass: 0.00, charge: 0)
  11: Si+ (mass: 0.00, charge: 1)
  12: Si (mass: 0.00, charge: 0)
  13: O+ (mass: 0.00, charge: 1)
  14: O (mass: 0.00, charge: 0)
  15: Si++ (mass: 0.00, charge: 2)
  16: OH (mass: 0.00, charge: 0)
  17: HOC+ (mass: 0.00, charge: 1)
  18: HCO+ (mass: 0.00, charge: 1)
  19: CO (mass: 0.00, charge: 0)
  20: CH (mass: 0.00, charge: 0)
  21: CH2 (mass: 0.00, charge: 0)
  22: C2 (mass: 0.00, charge: 0)
  23: HCO (mass: 0.00, charge: 0)
  24: H2O (mass: 0.00, charge: 0)
  25: O2 (mass: 0.00, charge: 0)
  26: H3+ (mass: 0.00, charge: 1)
  27: CH+ (mass: 0.00, charge: 1)
  28: CH2+ (mass: 0.00, charge: 1)
  29: CO+ (mass: 0.00, charge: 1)
  

In [3]:
# Show a few example reactions
for i, reaction in enumerate(network.reactions):
    reactants = " + ".join([f"{r.name}" for r in reaction.reactants])
    products = " + ".join([f"{p.name}" for p in reaction.products])
    print(f"  {i+1}: {reactants} -> {products}")
    print(f"      Rate: {reaction.get_sympy()}")
    print()

  1: H + e- -> H+ + e- + e-
      Rate: exp(-2.03914985e-6*lnTe**8 + 0.000111954395*lnTe**7 - 0.00263197617*lnTe**6 + 0.0348255977*lnTe**5 - 0.2877056*lnTe**4 + 1.56315498*lnTe**3 - 5.73932875*lnTe**2 + 13.536556*lnTe - 32.71396786)

  2: H+ + e- -> H
      Rate: 3.92e-13*invTe**0.6353

  3: H+ + e- -> H
      Rate: exp(-3.071135243196595e-9*lnTe**9 - 1.856767039775261e-8*lnTe**8 + 5.755614137575758e-7*lnTe**7 + 4.989108920299513e-6*lnTe**6 - 1.421502914054107e-5*lnTe**5 - 0.0003212605213188796*lnTe**4 - 0.002380861877349834*lnTe**3 - 0.02026044731984691*lnTe**2 - 0.7241125657826851*lnTe - 28.61303380689232)

  4: He + e- -> He+ + e- + e-
      Rate: exp(-3.64916141e-6*lnTe**8 + 0.000206723616*lnTe**7 - 0.0050090561*lnTe**6 + 0.0679539123*lnTe**5 - 0.56851189*lnTe**4 + 3.05803875*lnTe**3 - 10.7532302*lnTe**2 + 23.91596563*lnTe - 44.09864886)

  5: He+ + e- -> He
      Rate: 3.92e-13*invTe**0.6353

  6: He+ + e- -> He
      Rate: 1.54e-9*(1.0 + 0.3/exp(8.099328789667*invTe))/((Te**1.5*e

## Step 3: Generate C++ Kokkos Code

Now let's use the Builder to generate C++ code using the `kokkos_ode` template.

In [4]:
# Create builder and generate Kokkos C++ code
builder = Builder(network)
builds_dir = builder.build(template="kokkos_ode")

print("\nC++ Kokkos code generated successfully!")
print(f"Generated files are located at {builds_dir}")

Building network with template: kokkos_ode
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.hpp -> /Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.hpp
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.cpp -> /Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.cpp
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/CMakeLists.txt -> /Users/benwibking/amrex_codes/chemistry/jaff/examples/CMakeLists.txt
Network built successfully using template 'kokkos_ode'.
Output files are located in: /Users/benwibking/amrex_codes/chemistry/jaff/examples

C++ Kokkos code generated successfully!
Generated files are located at /Users/benwibking/amrex_codes/chemistry/jaff/examples


## Step 4: Examine Generated Files

Let's look at the structure and content of the generated files.

In [5]:
# List generated files
generated_files = [f for f in os.listdir(builds_dir) if f.endswith(('.cpp', '.hpp', '.txt'))]

print("Generated files:")
for file in sorted(generated_files):
    filepath = os.path.join(builds_dir, file)
    size = os.path.getsize(filepath)
    print(f"  {file:20} ({size:,} bytes)")

Generated files:
  CMakeLists.txt       (1,309 bytes)
  chemistry_ode.cpp    (6,000 bytes)
  chemistry_ode.hpp    (165,006 bytes)


## Step 5: Understanding the Generated Code Structure

Let's examine key components of the generated chemistry solver.

In [6]:
# Show the ChemistryODE class definition
with open(builds_dir + "/chemistry_ode.hpp", 'r') as f:
    lines = f.readlines()

# Find the class definition
class_start = None
for i, line in enumerate(lines):
    if "struct ChemistryODE" in line:
        class_start = i
        break

if class_start:
    print("ChemistryODE class structure:")
    print("=" * 50)
    # Show class definition and methods
    brace_count = 0
    for i in range(class_start, len(lines)):
        line = lines[i].rstrip()
        print(f"{i+1:2d}: {line}")
        
        if '{' in line:
            brace_count += line.count('{')
        if '}' in line:
            brace_count -= line.count('}')
            if brace_count == 0:
                break

ChemistryODE class structure:
13: struct ChemistryODE {
14:     // Number of species in the chemical network
15:     // PREPROCESS_NUM_SPECIES
16: 
17:     static constexpr int neqs = 37;
18: 
19:     // PREPROCESS_END
20: 
21:     // Species indices
22:     // PREPROCESS_COMMONS
23: 
24:     static constexpr int idx_H = 0;
25:     static constexpr int idx_e = 1;
26:     static constexpr int idx_Hj = 2;
27:     static constexpr int idx_He = 3;
28:     static constexpr int idx_Hej = 4;
29:     static constexpr int idx_Hejj = 5;
30:     static constexpr int idx_H2 = 6;
31:     static constexpr int idx_H2j = 7;
32:     static constexpr int idx_Hk = 8;
33:     static constexpr int idx_Cj = 9;
34:     static constexpr int idx_C = 10;
35:     static constexpr int idx_Sij = 11;
36:     static constexpr int idx_Si = 12;
37:     static constexpr int idx_Oj = 13;
38:     static constexpr int idx_O = 14;
39:     static constexpr int idx_Sijj = 15;
40:     static constexpr int idx_OH = 16;
41:    

In [7]:
# Show main function structure from chemistry_ode.cpp
with open(builds_dir + "/chemistry_ode.cpp", 'r') as f:
    lines = f.readlines()

# Find main function
main_start = None
for i, line in enumerate(lines):
    if "int main(" in line:
        main_start = i
        break

if main_start:
    print("Main function structure:")
    print("=" * 50)
    for i in range(main_start, len(lines)):
        line = lines[i].rstrip()
        print(f"{i+1:2d}: {line}")

Main function structure:
41: int main(int argc, char* argv[]) {
42:     Kokkos::initialize(argc, argv);
43: 
44:     {
45:         using execution_space = Kokkos::DefaultExecutionSpace;
46:         using scalar_type = double;
47:         using vec_type = Kokkos::View<scalar_type*, execution_space>;
48:         using mat_type = Kokkos::View<scalar_type**, execution_space>;
49: 
50:         ChemistryODE mySys{};
51: 
52:         // Time integration parameters
53:         const scalar_type t_start = 0.0;
54:         scalar_type t_end = 3.15576e9; // 100 years in seconds default
55: 
56:         // Parse command line arguments
57:         if (argc > 1) {
58:             t_end = std::stod(argv[1]);
59:         }
60: 
61:         std::cout << "Solving Chemistry ODE System\n";
62:         std::cout << "Number of species: " << mySys.neqs << "\n";
63:         std::cout << "Number of reactions: " <<
64: // PREPROCESS_NUM_REACTIONS
65: 
66: 287
67: 
68: // PREPROCESS_END
69:          << "\n";
70:

## Step 6: Building and Running Instructions

Here's how to build and run the generated C++ code:

* **Configure the build with CMake:**

In [8]:
!   cmake -S . -B build

-- Setting default Kokkos CXX standard to 17
-- Kokkos version: 4.7.0
-- The project name is: Kokkos
-- Using internal gtest for testing
-- Using -std=c++17 for C++17 standard as feature
-- Built-in Execution Spaces:
--     Device Parallel: NoTypeDefined
--     Host Parallel: NoTypeDefined
--       Host Serial: SERIAL
-- 
-- Architectures:
-- Using internal desul_atomics copy
-- Performing Test KOKKOS_LINK_OPTIONS_CHECK
-- Performing Test KOKKOS_LINK_OPTIONS_CHECK - Success
-- Experimental mdspan support is enabled
-- Using internal mdspan directory /Users/benwibking/amrex_codes/chemistry/jaff/examples/build/_deps/kokkos-src/tpls/mdspan/include
-- Kokkos Backends: SERIAL
-- The project name is: KokkosKernels

Kokkos Kernels version: 4.7.0
Kokkos Kernels ETI Types
   Devices:  <Serial,HostSpace>
   Scalars:  double
   Ordinals: int
   Offsets:  int
   Layouts:  LayoutLeft

Kokkos Kernels components
   COMMON:    ON
   BATCHED:   ON
   BLAS:      ON
   LAPACK:    ON
   GRAPH:     ON
   S

* **Build with parallel compilation:**

In [9]:
!   cmake --build build -j

[2/3] Building CXX object CMakeFiles/chemistry_ode.dir/chemistry_ode.cpp.oake
FAILED: [code=1] CMakeFiles/chemistry_ode.dir/chemistry_ode.cpp.o 
/usr/bin/c++ -DKOKKOS_DEPENDENCE -I/Users/benwibking/amrex_codes/chemistry/jaff/examples/build/_deps/kokkos-build -I/Users/benwibking/amrex_codes/chemistry/jaff/examples/build/_deps/kokkos-build/core/src -I/Users/benwibking/amrex_codes/chemistry/jaff/examples/build/_deps/kokkos-src/core/src -I/Users/benwibking/amrex_codes/chemistry/jaff/examples/build/_deps/kokkos-build/containers/src -I/Users/benwibking/amrex_codes/chemistry/jaff/examples/build/_deps/kokkos-src/containers/src -I/Users/benwibking/amrex_codes/chemistry/jaff/examples/build/_deps/kokkos-build/algorithms/src -I/Users/benwibking/amrex_codes/chemistry/jaff/examples/build/_deps/kokkos-src/algorithms/src -I/Users/benwibking/amrex_codes/chemistry/jaff/examples/build/_deps/kokkos-build/simd/src -I/Users/benwibking/amrex_codes/chemistry/jaff/examples/build/_deps/kokkos-src/simd/src -I/Us

* **Run the chemistry solver:**

In [10]:
!   ./build/chemistry_ode

zsh:1: no such file or directory: ./build/chemistry_ode


* **Or, run with custom time interval (1e12 seconds):**

In [11]:
!   ./build/chemistry_ode 1e12

zsh:1: no such file or directory: ./build/chemistry_ode
